## This is our workspace for statistical analysis!

Add cells, do exploratory analysis, go ham.

In [2]:
# feel free to add more

import sqlite3
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.tools import eval_measures

In [3]:
# Create connection to database
conn = sqlite3.connect('../data/db/avocado_project.db')
c = conn.cursor()

In [4]:
# Print out tables in database
list_tables_cmd = "SELECT name FROM sqlite_master WHERE type='table';"
c.execute(list_tables_cmd)
conn.commit()

for row in c:
  print(row)

('demographic',)
('hab2016',)
('hab2017',)
('hab2018',)


In [23]:
# Load all tables into dataframe
query1 = "SELECT * FROM demographic"
demo_df = pd.read_sql(query1, conn)

query2 = "SELECT * FROM hab2016"
hab2016_df = pd.read_sql(query2, conn)
hab2016_df.rename(columns={'units_current_year': '2016_units_current_year',
                          'dollars_current_year': '2016_dollars_current_year',
                          'asp_current_year': '2016_asp_current_year'}, inplace=True)

query3 = "SELECT * FROM hab2017"
hab2017_df = pd.read_sql(query3, conn)
hab2017_df.rename(columns={'units_current_year': '2017_units_current_year',
                          'dollars_current_year': '2017_dollars_current_year',
                          'asp_current_year': '2017_asp_current_year'}, inplace=True)

query4 = "SELECT * FROM hab2018"
hab2018_df = pd.read_sql(query4, conn)
hab2018_df.rename(columns={'units_current_year': '2018_units_current_year',
                          'dollars_current_year': '2018_dollars_current_year',
                          'asp_current_year': '2018_asp_current_year'}, inplace=True)

In [ ]:
mean_pop = []
median_pop = []
stddev_pop = []

for index, row in demo_df.iterrows():
    pop_list = [row['pop2016'], row['pop2017'], row['pop2018']]
    means = np.mean(pop_list)
    median = np.median(pop_list)
    std_dev = np.std(pop_list)
    mean_pop.append(means)
    median_pop.append(median)
    stddev_pop.append(std_dev)
    
demo_df["Mean Population"] = mean_pop
demo_df["Median Population"] = median_pop
demo_df["Standard Deviation Population"] = stddev_pop
demo_df

#we might want to remove the rows with age 999 as its just an aggregation of all the previous rows of that region


In [ ]:
region = []
current_units_mean = []
current_units_median = []
current_units_stddev = []

current_dollars_mean = []
current_dollars_median = []
current_dollars_stddev = []

current_asp_mean = []
current_asp_median = []
current_asp_stddev = []

for index, row in hab2016_df.iterrows(): 
    region.append(row['region'])
    for index1, row1 in hab2017_df.iterrows():
        for index2, row2 in hab2018_df.iterrows():
            current_units_list = [row['2016_units_current_year'], row1['2017_units_current_year'], row2['2018_units_current_year']]
            current_dollars_list = [row['2016_dollars_current_year'], row1['2017_dollars_current_year'], row2['2018_dollars_current_year']]
            current_asp_list = [row['2016_asp_current_year'], row1['2017_asp_current_year'], row2['2018_asp_current_year']]
            
    units_mean = np.mean(current_units_list)
    units_median = np.median(current_units_list)
    units_std_dev = np.std(current_units_list)
    current_units_mean.append(units_mean)
    current_units_median.append(units_median)
    current_units_stddev.append(units_std_dev)
            
    dollars_mean = np.mean(current_dollars_list)
    dollars_median = np.median(current_dollars_list)
    dollars_std_dev = np.std(current_dollars_list)
    current_dollars_mean.append(dollars_mean)
    current_dollars_median.append(dollars_median)
    current_dollars_stddev.append(dollars_std_dev)
            
    asp_mean = np.mean(current_asp_list)  
    asp_median = np.median(current_asp_list)
    asp_std_dev = np.std(current_asp_list)  
    current_asp_mean.append(asp_mean)
    current_asp_median.append(asp_median)
    current_asp_stddev.append(asp_std_dev)
            
combined_avo_data = {'region' : region,
                     'units_current_year_mean' : current_units_mean, 
                     'units_current_year_median' : current_units_median,
                    'units_current_year_std_dev' : current_units_stddev,
                    'dollars_current_year_mean' : current_dollars_mean, 
                     'dollars_current_year_median' : current_dollars_median,
                    'dollars_current_year_std_dev' : current_dollars_stddev,
                    'asp_current_year_mean' : current_asp_mean, 
                     'asp_current_year_median' : current_asp_median,
                    'asp_current_year_std_dev' : current_asp_stddev}

combined_avo_df = pd.DataFrame(combined_avo_data)
combined_avo_df

In [92]:
# Load all tables into dataframe
query1 = "SELECT * FROM demographic"
demo_df = pd.read_sql(query1, conn)
demo_df.rename(columns={'pop2016': '2016', 'pop2017': '2017', 'pop2018': '2018'}, inplace=True)

query2 = "SELECT * FROM hab2016"
hab2016_raw_df = pd.read_sql(query2, conn)
hab2016_raw_df['year'] = 2016

query3 = "SELECT * FROM hab2017"
hab2017_raw_df = pd.read_sql(query3, conn)
hab2017_raw_df['year'] = 2017

query4 = "SELECT * FROM hab2018"
hab2018_raw_df = pd.read_sql(query4, conn)
hab2018_raw_df['year'] = 2018

In [93]:
# demo_df.describe()

In [94]:
# parse out and sum rows that correspond to millenials
mill_df = demo_df.loc[(demo_df['age'] <= 35) & (demo_df['age'] >= 20)]
mill_df = mill_df.melt(id_vars=["region", "age", "sex"], var_name="year", value_name="millenial_pop")
mill_df = mill_df.groupby(by=['region','year'])
mill_df = mill_df.sum()
mill_df = mill_df.drop("age", axis=1)

# mill_df

In [98]:
# parse out and sum rows that correspond to boomers
boom_df = demo_df.loc[demo_df['age'] >= 35]
boom_df = boom_df.melt(id_vars=["region", "age", "sex"], var_name="year", value_name="boomer_pop")
boom_df = boom_df.groupby(by=['region','year'])
boom_df = boom_df.sum()
boom_df = boom_df.drop("age", axis=1)

# boom_df

In [112]:
combined_demo_df = mill_df.merge(boom_df, on=['region', 'year'], how='outer')
combined_demo_df = combined_demo_df.reset_index()
combined_demo_df = combined_demo_df.rename(columns={"level_0": "region", "level_1": "year"})
combined_demo_df['year']=combined_demo_df['year'].astype(int)
combined_demo_df

,region,year,millenial_pop,boomer_pop
0,California,2016,9181203,59361156
1,California,2017,9203129,59775868
2,California,2018,9206218,60149721
3,Great Lakes,2016,9950923,72346169
4,Great Lakes,2017,9956720,72529535
5,Great Lakes,2018,9963194,72704600
6,Midsouth,2016,8070600,59103583
7,Midsouth,2017,8119174,59596597
8,Midsouth,2018,8157019,60075594
9,Northeast,2016,12311019,88362289


In [113]:
# join all three avocado data frames
hab_df = pd.concat([hab2016_raw_df, hab2017_raw_df, hab2018_raw_df], axis=0)
hab_df['year'] = hab_df['year'].astype(int)
hab_df = hab_df.sort_values(by=['region'])
hab_df

,region,units_prior_year,units_current_year,dollars_prior_year,dollars_current_year,asp_prior_year,asp_current_year,year
0,California,2.815620e+08,2.742912e+08,2.938569e+08,3.161020e+08,12.553616,14.193095,2016
0,California,2.892980e+08,3.449431e+08,3.806062e+08,3.965100e+08,17.467113,15.074676,2018
0,California,3.163792e+08,3.102769e+08,3.574163e+08,4.020201e+08,16.156570,18.487848,2017
1,Great Lakes,1.565253e+08,1.585368e+08,1.746281e+08,1.920213e+08,13.370501,14.862985,2016
1,Great Lakes,1.767339e+08,2.205796e+08,2.455320e+08,2.528185e+08,18.420128,14.962134,2018
1,Great Lakes,1.828621e+08,1.892237e+08,2.183403e+08,2.597029e+08,17.024001,19.554727,2017
2,Midsouth,1.353713e+08,1.470024e+08,1.532692e+08,1.749785e+08,13.567583,14.537764,2016
2,Midsouth,1.626263e+08,2.020955e+08,2.206566e+08,2.370726e+08,17.740391,15.288319,2018
2,Midsouth,1.666395e+08,1.723995e+08,1.970757e+08,2.325388e+08,16.789138,18.956182,2017
3,Northeast,2.240129e+08,2.939438e+08,3.334003e+08,3.715844e+08,19.418093,16.582135,2018


In [114]:
# hab_df.describe()

In [117]:
# load in per cap avocado data
file = "../data/csv/avo_per_cap.csv"
per_cap_df = pd.read_csv(file)
per_cap_df

,region,year,total_avo_sold,total_pop,per_cap_avo_sold
0,California,2016,274291200,39051551,7.02
1,Great Lakes,2016,158536800,46772128,3.39
2,Midsouth,2016,147002400,38131989,3.86
3,Northeast,2016,207781600,56680619,3.67
4,Plains,2016,88054250,21118963,4.17
5,South Central,2016,279874700,39376454,7.11
6,Southeast,2016,175557200,43548895,4.03
7,West,2016,282185500,35087768,8.04
8,California,2017,310276900,39238620,7.91
9,Great Lakes,2017,189223700,46832264,4.04


In [119]:
# join combined demographic and avo data
joined_df = combined_demo_df.merge(per_cap_df, on=['region', 'year'], how='outer')
joined_df = joined_df.merge(hab_df, on=['region', 'year'], how='outer')
joined_df

,region,year,millenial_pop,boomer_pop,total_avo_sold,total_pop,per_cap_avo_sold,units_prior_year,units_current_year,dollars_prior_year,dollars_current_year,asp_prior_year,asp_current_year
0,California,2016,9181203,59361156,274291200,39051551,7.02,2.815620e+08,2.742912e+08,2.938569e+08,3.161020e+08,12.553616,14.193095
1,California,2017,9203129,59775868,310276900,39238620,7.91,3.163792e+08,3.102769e+08,3.574163e+08,4.020201e+08,16.156570,18.487848
2,California,2018,9206218,60149721,344943100,39400984,8.75,2.892980e+08,3.449431e+08,3.806062e+08,3.965100e+08,17.467113,15.074676
3,Great Lakes,2016,9950923,72346169,158536800,46772128,3.39,1.565253e+08,1.585368e+08,1.746281e+08,1.920213e+08,13.370501,14.862985
4,Great Lakes,2017,9956720,72529535,189223700,46832264,4.04,1.828621e+08,1.892237e+08,2.183403e+08,2.597029e+08,17.024001,19.554727
5,Great Lakes,2018,9963194,72704600,220579600,46885784,4.70,1.767339e+08,2.205796e+08,2.455320e+08,2.528185e+08,18.420128,14.962134
6,Midsouth,2016,8070600,59103583,147002400,38131989,3.86,1.353713e+08,1.470024e+08,1.532692e+08,1.749785e+08,13.567583,14.537764
7,Midsouth,2017,8119174,59596597,172399500,38407084,4.49,1.666395e+08,1.723995e+08,1.970757e+08,2.325388e+08,16.789138,18.956182
8,Midsouth,2018,8157019,60075594,202095500,38665871,5.23,1.626263e+08,2.020955e+08,2.206566e+08,2.370726e+08,17.740391,15.288319
9,Northeast,2016,12311019,88362289,207781600,56680619,3.67,1.911689e+08,2.077816e+08,2.342339e+08,2.654976e+08,14.719099,15.702439


In [122]:
# save joined_df (master joined table) to sqlite db
joined_df.to_sql('joined_data', conn, if_exists='replace', index=False)

# test that data was saved properly
query = "SELECT * FROM joined_data"
df = pd.read_sql(query, conn)
df

,region,year,millenial_pop,boomer_pop,total_avo_sold,total_pop,per_cap_avo_sold,units_prior_year,units_current_year,dollars_prior_year,dollars_current_year,asp_prior_year,asp_current_year
0,California,2016,9181203,59361156,274291200,39051551,7.02,2.815620e+08,2.742912e+08,2.938569e+08,3.161020e+08,12.553616,14.193095
1,California,2017,9203129,59775868,310276900,39238620,7.91,3.163792e+08,3.102769e+08,3.574163e+08,4.020201e+08,16.156570,18.487848
2,California,2018,9206218,60149721,344943100,39400984,8.75,2.892980e+08,3.449431e+08,3.806062e+08,3.965100e+08,17.467113,15.074676
3,Great Lakes,2016,9950923,72346169,158536800,46772128,3.39,1.565253e+08,1.585368e+08,1.746281e+08,1.920213e+08,13.370501,14.862985
4,Great Lakes,2017,9956720,72529535,189223700,46832264,4.04,1.828621e+08,1.892237e+08,2.183403e+08,2.597029e+08,17.024001,19.554727
5,Great Lakes,2018,9963194,72704600,220579600,46885784,4.70,1.767339e+08,2.205796e+08,2.455320e+08,2.528185e+08,18.420128,14.962134
6,Midsouth,2016,8070600,59103583,147002400,38131989,3.86,1.353713e+08,1.470024e+08,1.532692e+08,1.749785e+08,13.567583,14.537764
7,Midsouth,2017,8119174,59596597,172399500,38407084,4.49,1.666395e+08,1.723995e+08,1.970757e+08,2.325388e+08,16.789138,18.956182
8,Midsouth,2018,8157019,60075594,202095500,38665871,5.23,1.626263e+08,2.020955e+08,2.206566e+08,2.370726e+08,17.740391,15.288319
9,Northeast,2016,12311019,88362289,207781600,56680619,3.67,1.911689e+08,2.077816e+08,2.342339e+08,2.654976e+08,14.719099,15.702439
